In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from torchvision.datasets import MNIST
import torch
from tralo import TrainingLogger
import random
import time
import numpy as np
from os.path import expanduser

from torch import nn
from torch.utils.data import DataLoader

configs = [
    dict(a=1, b='Muh'),
]


In [ ]:

def random_metric(model):
    return [['abc', 0.3]]

with TrainingLogger(metric=(random_metric, 10)) as logger:

    for i_epoch in range(2):
        for i_iter in range(100):

            fake_loss = np.random.normal() + 3 - (i_epoch*100 + i_iter) / 1000
            logger(i_epoch*100 + i_iter, loss=fake_loss)

        # let's assume we evaluate the metric only once per epoch
        fake_metric = np.random.normal() + (i_epoch*100 + i_iter) / 1000
        logger(i_epoch*100 + i_iter, fake_metric=fake_metric)


In [ ]:
set(x for k in logger.metric_values.values() for x in k.keys())

In [ ]:
logger.plots()

In [ ]:
from torchvision.datasets import MNIST
from os.path import expanduser

import torch
from torch import nn
from torch.nn import functional as nnf


from torchvision import transforms

tf = transforms.Compose([
    transforms.ToTensor(),
])

mnist = MNIST(expanduser('~/datasets'), transform=tf, download=True)

data_loader = torch.utils.data.DataLoader(mnist, batch_size=16)

model = nn.Sequential(
    nn.Conv2d(1, 4, 3),
    nn.MaxPool2d(2),
    nn.Conv2d(4, 8, 3),
    nn.MaxPool2d(2),
    nn.Conv2d(8, 16, 3),
    nn.MaxPool2d(3),
    nn.Conv2d(16, 10, 1),
    nn.Flatten()
)

opt = torch.optim.Adam(model.parameters(), lr=0.001)


### File-based version

This will save statistics in `logs/mnist_run`.

In [ ]:
with TrainingLogger(f'mnist_run2', interval=50, utilization_iters=400, grad_weights=[('second_layer', model[2])], grad_interval=20) as logger:
    i = 0

    # make sure we write utility to disk more often
    logger.utilization_process.step_interval.value = 5

    for i_epoch in range(1):
        for data_x, data_y in data_loader:

            pred = model(data_x)
            loss = nnf.cross_entropy(pred, data_y)

            opt.zero_grad()
            loss.backward()
            opt.step()

            logger.iter(i=i, loss=loss)
            i += 1
            
            if i > 500:
                break

In [ ]:
logger.plots() 

### In-memory version

In [ ]:
with TrainingLogger(interval=50, utilization_iters=500, grad_weights=[('second_layer', model[2])], grad_interval=20) as logger:
    i = 0
    for i_epoch in range(1):
        for data_x, data_y in data_loader:

            # make sure we write utility to disk more often
            logger.utilization_process.step_interval.value = 20

            pred = model(data_x)
            loss = nnf.cross_entropy(pred, data_y)

            opt.zero_grad()
            loss.backward()
            opt.step()

            logger.iter(i=i, loss=loss)
            i += 1
            
            if i > 500:
                break

In [ ]:
log.plots()